<a href="https://colab.research.google.com/github/jazkre/01ZLMA/blob/main/R/01ZLMA_ex08_Binary_Data_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 08 of the course 01ZLMA. 

# GLM for Discrete response - Binary Data Analysis (02)

Alternative and Binomial responses

**Bernoulli (Alternative) Model**

$$Y_{i,j} \sim Be(\pi_i) \ i = 1,\ldots,K \ \text{and} \ j = 1,\ldots, n_i.$$
$K$ is number of groups, $n_i$ is number of observations in group $i$ and $\sum_{i=1}^{K} = N$
$$ E[Y_{i,j}] = \pi_i \ \text{and} \ g(\pi_i) = \eta_i =x_i^T \beta $$


**Binomial Model**
$$Y_i = \sum_{j=1}^{n_i} Y_{i,j} \sim Bi(n_i, \pi_i)$$

**Without continuos covariate (only factor variables)**

$K$ is constant and $n_i \rightarrow \infty $

**With at least one continuos covariate**

$n_i \approx 1$ ( $n_i$ is small enough) and $K \rightarrow \infty$



## Link functions for binary data

**Logistic function:**

The logistic function is the canonical link function for binary responses, and it is CDF of the standard logistic distribution.

$$\pi_i = \frac{1}{1+e^{-x_i^T \beta}} $$ 


**Probit function:**

The CDF of the normal distribution. 
$$\pi_i = \Phi({x_i^T \beta}) $$ 


**Cauchit function:**

The CDF of the Cauchy distribution

$$\pi_i = \frac{1}{\pi}\text{arctan}(x_i^T \beta) + \frac{1}{2} $$ 


**Complementary log-log (cloglog) function:**

The inverse of the conditional log-log function (CDF of the Gumbel distribution)

$$\pi_i = 1 − e^{-e^{x_i^T \beta}}$$

The counter part of the cloglog function is log-log link function.

In [ ]:
library(tidyverse)
install.packages("latex2exp")
library(latex2exp)
library(ggplot2)
install.packages("gridExtra")
library(gridExtra)
install.packages("GGally")
library(GGally)

#library(Matrix)
#library(MASS)

In [ ]:
map(c("logit", "probit", "cauchit", "cloglog"),  make.link) %>%
map_df(
  function(link) {
    tibble(x = seq(-5, 5, length.out = 101),
           y = link$linkinv(x),
           link_name = link$name)
  }
  ) %>%
  ggplot(aes(x = x, y = y, colour = link_name)) +
  geom_line()+
  ylab(TeX("$\\pi_i$"))+
  xlab(TeX("$x_i^T \\beta$"))

## Logistic regression with Titanic dataset

https://www.kaggle.com/c/titanic/data

| Variable |                 Definition                 |                       Key                      |
|:--------:|:------------------------------------------:|:----------------------------------------------:|
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |
| ticket   | Ticket number                              |                                                |
| fare     | Passenger fare                             |                                                |
| cabin    | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

In [ ]:
install.packages("titanic")
library(titanic)
knitr::kable(summary(titanic_train))

In [ ]:
knitr::kable(summary(titanic_test))

# Fast and furios solution of the HW from the previous Ex07

Consider a model with continuos variables `age`, `fare`, and any factor variable. 

* Create factor `age`, which takes values 1 (child) and 0 (adult).
* Create factor from varaible `fare` by spliting into 10 groups with approx same number of observations.
* What percentage is the chance of survival lower for adult compare to child? 
* Depends the probability of survival on fare? Test it.
* Assume that the chance of survival increases with exponential increasig fare. How the chance of survival increased if the person spent an extra 10 pound for a ticket? 
* Build a model where the probabilty of survival depends on both `age` and `fare`. Are both covariates significant?
* 

In [ ]:
? quantile

In [ ]:
data_HW <- titanic_train %>%
  dplyr::select(Survived,Age,Fare,Pclass,Sex,Embarked)

data_HW %>% mutate_if(is.character,as.factor) %>% summary() %>%  knitr::kable()

data_HW <- data_HW %>%
  filter(Embarked %in% c("C","Q","S")) %>%
  transmute(survived = as.factor(Survived),
            survived_num = Survived,
            age  = cut(Age, breaks=c(-Inf, 16, Inf), labels=c("child","adult")),
            age_num = Age,
            fare = Fare,
            fare_quantile = ntile(Fare, 8),
            fare_quantile2 = cut(Fare, quantile(Fare, prob = seq(0,1,length = 9), type = 8)), 
            class = as.factor(Pclass),
            sex = as.factor(Sex),
            embarked = as.factor(Embarked)) %>%
  drop_na()          

data_HW %>% summary() %>%  knitr::kable()       
str(data_HW)


In [ ]:
options(repr.plot.width=16, repr.plot.height=12)
ggpairs(data_HW)

In [ ]:
# Simple Logistic Regression model with all discrete covariates fare, sex, and age, but without interactions
mod_HW1=glm(survived~age+sex+fare_quantile2,family=binomial(link = "logit"),data_HW) #
summary(mod_HW1)

fare_quantile2(7.75,7.9] len po 15 centoch ⛎


In [ ]:
#Function to estimate OR with lower and upper limit of 95% CI for OR
OR_coef <- function(variable,model,CI){
  param <- coef(model)
  where <- grep(variable,names(param))[1]
  beta  <- param[where]
  se <- summary(model)$coef[where,2]
  or <- exp(beta)
  ci <- exp(beta+c(-1,1)*qnorm(CI/2+0.5)*se)
  out <- data.frame(ci[1],or, ci[2])
  names(out) <- c("LCL","OR","UCL")
  out
}
OR_coef("age",mod_HW1,0.95)

The chances of drowning were almost twice as high in the group of adults as in the group of children. 

In [ ]:
# Check model with factorized Fare
mod_HW0=glm(survived~age+sex,family=binomial(link = "logit"),data_HW) #
mod_HW1=glm(survived~age+sex+fare_quantile2,family=binomial(link = "logit"),data_HW) #
mod_HW2=glm(survived~age+sex+fare,family=binomial(link = "logit"),data_HW) #
summary(mod_HW1)


Yes, covariate `fare` is statisticaly significant in the model. But it's better to test it with Deviation test (LRT) compare to Wald tests from summary() function.

In [ ]:
anova(mod_HW1,mod_HW0,test="Chisq")
drop1(mod_HW1,test="Chisq")

Confirmed.

In [ ]:
# Fare as continuous variable
mod_HW3=glm(survived~I(fare/10),family=binomial(link = "logit"),data_HW) #

mod_HW4=glm(survived~I(fare/10)+sex+age,family=binomial(link = "logit"),data_HW) #
#summary(mod_HW4)
exp(coef(mod_HW3))
exp(coef(mod_HW4))

The chance of survival increased by 12.9% with the ticket price 10 pounds higher.

Question: What can you say about Causality? Does a higher price result in a greater chance of survival?



In [ ]:
# Model without sex variable
mod_HW5=glm(survived~I(fare/10)+I(age_num/10),family=binomial(link = "logit"),data_HW) #
summary(mod_HW5)
exp(coef(mod_HW5))


## Binomial data approach

In [ ]:
#data_HW

In [ ]:
data_HW_bin <- data_HW %>%
  group_by(sex,age,fare_quantile2) %>%#všetky premenne vezmeme ako faktorove a možeme spraviť kontingenčnu tabulku.
  summarise(n = n(),
         m = sum(survived_num)
        )
 data_HW_bin        

jak by vyzeral saturovaný odhad? 

female	child	(31,69.3]	12	6 -> v danej skupine ? v každej skupive v podstate hádzeme mincou -> 6/12 (psť prežitia v skupine)

 n-> ∞ , m -> ∞

 



### Some additional questions 

**Estimate** null model and compare obtained probality of survived with estimated probability from survived-not survived ratio.

In [ ]:
mod_0 = glm(cbind(m,n-m)~1,family=binomial(link="logit"),data = data_HW_bin)
summary(mod_0) #nulový model len str hodnota
invlogit = function(x) return(exp(x)/(1+exp(x))) # inverse logit function
p0_model = invlogit(coef(mod_0))
p0_model #psť prezitia 0.407

In [ ]:
# Probability of survive
p0_stat = sum(data_HW_bin$m)/sum(data_HW_bin$n)
print(cbind(p0_model,p0_stat))


In [ ]:
# Chance of survive
exp(coef(mod_0))
p0_stat/(1-p0_stat) #napriamo psť, že človek prežije / psť, že neprežije

Estimate model with one covariate factorized `fare` and compare OR computed directly from estimated regression coefficient with OR compputed from probabilities of survived for each group.

In [ ]:
mod_fare = glm(cbind(m,n-m)~fare_quantile2,family=binomial(link="logit"),data = data_HW_bin)
#summary(mod_fare)
anova(mod_fare,mod_0,test="Chisq") #porovname
# Fare price is highly significant in the model.
exp(coef(mod_fare)) %>% knitr::kable()




(Intercept)               |  0.2459016



Question: Assume you spent 25 pounds for a ticket. What is your chance to survive compare to someone who spent only 5 pounds?

In [ ]:
# Compute probability of survival:
invlogit(coef(mod_fare)[1]) 
invlogit(coef(mod_fare)[1]+coef(mod_fare)[-1]) %>% knitr::kable()


psti prezitia pre jednotlive skupiny

In [ ]:
# Check: odds ratio computed by hands.
Pr_1<- (invlogit(coef(mod_fare)[1])) #\1.skupina
Pr_8 <- (invlogit(coef(mod_fare)[1] +coef(mod_fare)[8]))#8.skupina
Pr_1; Pr_8

In [ ]:
OR_1 <- Pr_1/(1-Pr_1)
OR_8 <- Pr_8/(1-Pr_8) 
OR_1; OR_8

In [ ]:
OR_8/OR_1 #osma a prva cenova - pomer 10x vyššia šanca pre 8
(exp(coef(mod_fare)[8]) == OR_8/OR_1)


**Estimate saturated model**

If we have  fixed number of parameters,  saturated model obtain every  "observation" and residual deviance of this saturated model is "zero". 


pre kazdy riadok z tej kontingencnej tabulky mame vlastne teraz odhad:

In [ ]:
mod_HW_sat=glm(cbind(m,n-m)~fare_quantile2*age*sex,family=binomial(link = "logit"),data_HW_bin) #
summary(mod_HW_sat)

Residual deviance: 3.1965e-10 -> deviacna statistika teda 0


We can test other models against saturated one.

In [ ]:
mod_HW_bin1=glm(cbind(m,n-m)~fare_quantile2+age+sex,family=binomial(link = "logit"),data_HW_bin) #
summary(mod_HW_bin1)

In [ ]:
drop1(mod_HW_sat, test="Chisq")
anova(mod_HW_sat,mod_HW_bin1, test="Chisq")


## Question:
* How do you interpret previous result?
* Is simplier aditive model without interactions good enough?

vacsinou nema zmysel sa tym zaoberat podrobne? 

## Residual analysis

In [ ]:
# start with simple model

mod_1=glm(survived~age_num+sex+class,family=binomial(link = "logit"),data_HW) #
summary(mod_1)

#mod_1=glm(survived~sex +fare,family=binomial(link = "logit"),data_HW) #
#summary(mod_1)


In [ ]:
res_resp <- residuals(mod_1, type="response")
res_work <- residuals(mod_1, type="working")
res_pear <- residuals(mod_1, type="pearson")
#cbind(res_resp,res_work,res_pear)

By default, residuals.glm defaults to `type="deviance"`. A perfect fit of a datapoint (which never occurs) gives a deviance of zero as "log(1) =0". A poorly fitting point has a large residual deviance.

In [ ]:
par(mfrow=c(1,2))
plot(mod_1)

All plots look funny. What do you think, is there any problem in the model?


* The **Residuals vs Fitted** plot can help you see, for example, if there are curvilinear trends that you missed. But the fit of a logistic regression is curvilinear by nature, so you can have odd looking trends in the residuals with nothing amiss.
* The **Normal Q-Q** plot helps you detect if your residuals are normally distributed. But the deviance residuals don't have to be normally distributed for the model to be valid, so the normality / non-normality of the residuals doesn't necessarily tell you anything.
* The **Scale-Location** plot can help you identify heteroscedasticity. But logistic regression models are pretty much heteroscedastic by nature.
* The **Residuals vs Leverage** can help you identify possible outliers. But outliers in logistic regression don't necessarily manifest in the same way as in linear regression, so this plot may or may not be helpful in identifying them.

source:https://stats.stackexchange.com/questions/121490/interpretation-of-plot-glm-model


In [ ]:
plot(mod_1,which=1)

In [ ]:
plot(data_HW$age_num,residuals(mod_1),col=c("blue","red")[1+data_HW$survived_num])
lines(lowess(data_HW$age_num,residuals(mod_1)),col="black",lwd=2)
lines(lowess(data_HW$age_num[data_HW$survived_num==0],residuals(mod_1)[data_HW$survived_num==0]),col="blue")
lines(lowess(data_HW$age_num[data_HW$survived_num==1],residuals(mod_1)[data_HW$survived_num==1]),col="red")
abline(h=0,lty=2,col="grey")

In [ ]:
data_HW_mod <- data_HW %>%
  mutate(fitted = mod_1$fitted.values,
         #fitted = predict(mod_1, newdata = data_HW, type = "response"),
         predict = as.factor(ifelse(fitted > 0.5, "1", "0")),
         r_deviance_std =rstandard(mod_1, type = "deviance"),
         r_pearson_std = rstandard(mod_1, type = "pearson") )
head(data_HW_mod)

In [ ]:
p1 <- ggplot(data_HW_mod, aes(x = age_num,y = fitted, color = sex, shape = class)) + 
      geom_point() 
p2 <- ggplot(data_HW_mod, aes(x = age_num,y = r_deviance_std, color = sex, shape = class)) + 
      geom_point() 
grid.arrange(p1, p2, nrow = 1)

toto by mohlo by napomocne (dolny image) aby sme videli ako sa model chová, resp kde najviac chybujme

In [ ]:
p1 <- ggplot(data_HW_mod, aes(x = age_num,y = fitted, color = survived)) + 
      geom_point() 
p2 <- ggplot(data_HW_mod, aes(x = age_num,y = r_deviance_std, color = survived)) + 
      geom_point() 
grid.arrange(p1, p2, nrow = 1)

## Post hoc analysis by quality of fitted values

In [ ]:
# Fit a  model
table(data_HW_mod$predict, data_HW$survived)

In [ ]:
install.packages("caret")
library(caret)
install.packages("e1071")
library(e1071)
install.packages("tidymodels")
library(tidymodels)
 

In [ ]:
confusionMatrix(data_HW_mod$predict, data_HW_mod$survived, positive = "1")

Accuracy -> neni dobra metrika

recall alebo precision lepšie

In [ ]:
ggplot(data_HW_mod, aes(x=fitted, color=survived)) +
  geom_histogram(fill="white",aes(y=..density..), alpha=0.1, position="identity") +
   geom_density(aes(y=..density..))

# Overlaid histograms
#ggplot(df, aes(x=weight, color=sex)) +
#  geom_histogram(fill="white", alpha=0.5, position="identity")

rezneme to na 0.4

In [ ]:
data_HW_mod <- data_HW_mod %>%
  mutate(predict_04 = as.factor(ifelse(fitted > 0.4, "1", "0")) )
head(data_HW_mod)
confusionMatrix(data_HW_mod$predict_04, data_HW_mod$survived, positive = "1")

lepšie predikujeme preživších ale horžie tých nepreživších, keď ten trashhold posunieme na 0.4. 



In [ ]:
(332+232)/(332 + 55 + + 86 + 232) # Accuracy from Confusion Matrix

## Use test data

In [ ]:
head(titanic_test)

data sme upravovali, tak aj tie testovacie musime upraviť rovnakym sposobom!!! inak by sa to dosralo

In [ ]:
data_test <- titanic_test %>%
  dplyr::select(Age,Fare,Pclass,Sex,Embarked) %>%
  filter(Embarked %in% c("C","Q","S")) %>%
  drop_na %>%
  transmute(age  = cut(Age, breaks=c(-Inf, 16, Inf), labels=c("child","adult")),
            age_num = Age,
            fare = Fare,
            fare_quantile = ntile(Fare, 8),
            fare_quantile2 = cut(Fare, quantile(Fare, prob = seq(0,1,length = 9), type = 8)), 
            class = as.factor(Pclass),
            sex = as.factor(Sex),
            embarked = as.factor(Embarked)) %>%
  drop_na 
  summary(data_test)

In [ ]:
## Plot train

data_test <- data_test %>%
  mutate(fitted = predict(mod_1, newdata = data_test, type = "response"),#response! modelujeme odozvu!!!
         predict = as.factor(ifelse(fitted > 0.5, "1", "0")) ) %>%
         arrange(age_num)
head(data_test)


plt <- ggplot(data_test, aes(x = age_num, y = fitted, colour = sex:class)) +
    geom_line() +
    geom_rug(aes(y = survived_num, colour = survived), data = data_HW_mod) +
    #scale_colour_discrete(name = 'Survived') +
    labs(x = 'Age', y = 'Probability of survived')
plt



In [ ]:
data_test_select <- data_test %>%
  dplyr::filter(sex == "male",
               class == "2")

plt <- ggplot(data_test_select, aes(x = age_num, y = fitted)) +
    geom_line() +
    geom_rug(aes(y = survived_num, colour = survived), data = data_HW_mod) +
    #scale_colour_discrete(name = 'Survived') +
    labs(x = 'Age', y = 'Probability of survived')
plt


Wrong way to plot confidence interval

In [ ]:
#plt <- ggplot(data_test, aes(x = age_num, y = fitted, colour = sex:class)) +
#    geom_line() +
#    geom_rug(aes(y = survived_num, colour = survived), data = data_HW_mod) +
#    #scale_colour_discrete(name = 'Survived') +
#    labs(x = 'Age', y = 'Probability of survived')
#plt

In [ ]:
## add standard errors and compute a 95% interval the wrong way
data_test_select <- data_test_select %>%
  mutate(wrong_se = predict(mod_1, newdata = data_test_select, type = 'response',se.fit = TRUE)$se.fit,
         wrong_upr = fitted + (2 * wrong_se),
         wrong_lwr = fitted - (2 * wrong_se))

plt + geom_ribbon(data = data_test_select, aes(ymin = wrong_lwr, ymax = wrong_upr),
                  alpha = 0.1)




In [ ]:
#prečo je to wrong way? 
#konf interval zasahuje do zapornych hodnot ->zaporna psť a to je blbosť -_-
  #odozva +- rozptyl

Right way

In [ ]:
#predict(mod_1, data_test_select, se.fit = TRUE)


In [ ]:
ilink <- family(mod_1)$linkinv
quant_t <- qt(0.025, df = df.residual(mod_1), lower.tail = FALSE)
quant_n <- qnorm(0.005, lower.tail = FALSE)
## add fit and se.fit on the **link** scale
data_test_select <- data_test_select %>%
   mutate(fit_link = predict(mod_1, data_test_select, se.fit = TRUE)$fit,
          se_link  = predict(mod_1, data_test_select, se.fit = TRUE)$se.fit,
          fit  = ilink(fit_link),
          fit_ul = ilink(fit_link + (quant_n * se_link)), #najprv kvantil, potom až do fitu??? or sth like that
          fit_ll = ilink(fit_link - (quant_n * se_link)))
## show
head(data_test_select)

plt + geom_ribbon(data = data_test_select,
                  aes(ymin = fit_ll, ymax = fit_ul),
                  alpha = 0.1)



In [ ]:
#už neni symetr.
#
#

Binary classification

https://en.wikipedia.org/wiki/Binary_classification

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

https://en.wikipedia.org/wiki/Receiver_operating_characteristic



In R many different packages:

https://rviews.rstudio.com/2019/03/01/some-r-packages-for-roc-curves/

https://cran.r-project.org/web/packages/ROCit/index.html

https://cran.r-project.org/web/packages/pROC/index.html

https://cran.r-project.org/web/packages/ROCR/

In [ ]:
install.packages("pROC")
library(pROC)

Intro with pROC: https://rpubs.com/Wangzf/pROC

In [ ]:
plot.roc(data_HW_mod$survived, data_HW_mod$fitted,          # data
         percent = TRUE,                    # show all values in percent
         #partial.auc=c(100, 90), 
         #partial.auc.correct=True,          # define a partial AUC (pAUC)
         print.auc=TRUE,                    
         #display pAUC value on the plot with following options:
         #print.auc.pattern = "Corrected pAUC (100-90%% SP):\n%.1f%%",
         #print.auc.col = "#1c61b6",
         #auc.polygon = TRUE, 
         #auc.polygon.col = "#1c61b6",       # show pAUC as a polygon
         #max.auc.polygon = TRUE, 
         #max.auc.polygon.col = "#1c61b622", # also show the 100% polygon
         main = "Partial AUC (pAUC)")



max plocha pod auc krivkou? 1

0.5 pod krivkou -> keď náhodne si vyberám 0 alebo 1 -> je to zle, model zlý -> nad 0.9 tak kvalitné klasifikátory



skoky: musia odpovedat poctu dat? specificita sv sensitivita ->

jak zistíme ten 1 bod na nej? tam jak sme volili ten trashold 0.4, to je náš parameter, tym budeme posuvať a zakaždym dostaneme confusion mtx (CF_M) a z toho vieme vypočitat secificitu a senzitivitu (ku každej CF_M) a na zaklade toho vynašame tern AUC graf - krivku

kolko je tam moznuch bodov? maximálne toľko koľko je dat. ale može sa stať, že predikujeme tovnako pre rozne trasholdy.

In [ ]:
plot.roc(data_HW_mod$survived, data_HW_mod$fitted,          # data
         percent = TRUE,                    # show all values in percent
         partial.auc=c(100, 90), 
         partial.auc.correct=TRUE,          # define a partial AUC (pAUC)
         print.auc=TRUE,                    
         #display pAUC value on the plot with following options:
         print.auc.pattern = "Corrected pAUC (100-90%% SP):\n%.1f%%",
         print.auc.col = "#1c61b6",
         auc.polygon = TRUE, 
         auc.polygon.col = "#1c61b6",       # show pAUC as a polygon
         max.auc.polygon = TRUE, 
         max.auc.polygon.col = "#1c61b622", # also show the 100% polygon
         main = "Partial AUC (pAUC)")
plot.roc(data_HW_mod$survived, data_HW_mod$fitted,
         percent = TRUE, 
         add = TRUE, 
         type = "n",                        # add to plot, but don't re-add the ROC itself (useless)
         partial.auc = c(100, 90), 
         partial.auc.correct = TRUE,
         partial.auc.focus = "se",          # focus pAUC on the sensitivity
         print.auc = TRUE, 
         print.auc.pattern = "Corrected pAUC (100-90%% SE):\n%.1f%%", 
         print.auc.col = "#008600",
         print.auc.y = 40,                  # do not print auc over the previous one
         auc.polygon = TRUE, 
         auc.polygon.col = "#008600",
         max.auc.polygon = TRUE, 
         max.auc.polygon.col = "#00860022")

In [ ]:
install.packages("ROCR")
library(ROCR)
pr <- prediction(data_HW_mod$fitted, data_HW_mod$survived)
prf <- performance(pr, measure = "tpr", x.measure = "fpr")
plot(prf)
#? performance
auc <- performance(pr, measure = "auc")
auc <- auc@y.values[[1]]
auc

## Your Turn :

* Compute a new variable `nrelatives = sibsp + parch`
* Fit model with this new variable (you can try use it as continuous or as factor (change levels)) and `sex, age, fare, class, embarked`
* Try to use `step` function to find the "best" model.
* Validate the model. Are all variables significant? If you have any numerical variable, does it need any transformation?
* Plot densities of fitted probabilities for survived/not survived and choose optimal threshold.
* Compute the confusion matrix and basic statistics and compare results of your model with model mod_1
* Plot ROC curve of your model together with ROC curve of mod_1. Which one has larger AUC.

For fun: ChatGPT approach with prompt:

I need help with analysis in R. Please use the titanic dataset from install.packages("titanic"), library(titanic) and help me build a logistic regression model to estimate the survival probability from training data and estimate the probability in the test data. Use tidymodels to do this.

In [ ]:
library(tidyverse)
library(tidymodels)
library(titanic)

data("titanic_train")
data("titanic_test")

set.seed(123)
titanic_split <- initial_split(titanic_train, prop = 0.8)
train_data <- training(titanic_split)
test_data <- testing(titanic_split)

# Convert the outcome variable to a factor
train_data <- train_data %>%
  mutate(Survived = as.factor(Survived))

test_data <- test_data %>%
  mutate(Survived = as.factor(Survived))

# Create the recipe
rec <- recipe(Survived ~ Pclass + Sex + Age + SibSp + Parch + Fare + Embarked, data = train_data) %>%
  step_dummy(all_nominal(), -all_outcomes()) %>%
  step_center(all_numeric(), -all_outcomes()) %>%
  step_scale(all_numeric(), -all_outcomes()) %>%
  step_impute_knn(all_predictors())


In [ ]:
logistic_model <- logistic_reg() %>%
  set_engine("glm") %>%
  set_mode("classification")


In [ ]:
# Create the workflow
titanic_workflow <- workflow() %>%
  add_recipe(rec) %>%
  add_model(logistic_model)

# Train the model
titanic_fit <- titanic_workflow %>%
  fit(data = train_data)

# Estimate probabilities of survival for the test dataset
test_results <- titanic_fit %>%
  predict(test_data, type = "prob")

# Calculate AUC-ROC score
test_results %>%
  bind_cols(test_data) %>%
  roc_auc(Survived, .pred_0)

In [ ]:
install.packages("precrec")
library(precrec)

# Bind the predictions to the test_data
results_with_actuals <- test_results %>%
  bind_cols(test_data)

# Calculate ROC curve points
roc_curve_points <- roc_curve(actuals = results_with_actuals$Survived, scores = results_with_actuals$.pred_1)

# Plot ROC curve
autoplot(roc_curve_points)